<a href="https://colab.research.google.com/github/WoojinJeonkr/DeepLearning/blob/main/Text_Classification_With_Scratch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Scratch를 통한 텍스트 분류
- 내용 출처: https://keras.io/examples/nlp/text_classification_from_scratch/

## 01. 라이브러리 설정

In [1]:
import tensorflow as tf
import numpy as np

## 02. 데이터 다운로드 및 확인
- 사용 데이터: [IMDB 영화 리뷰](ttps://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz)

In [2]:
# 데이터 다운로드
!curl -O https://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz
!tar -xf aclImdb_v1.tar.gz

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 80.2M  100 80.2M    0     0  18.3M      0  0:00:04  0:00:04 --:--:-- 18.3M


In [3]:
# 폴더 생성(train 데이터와 test 데이터 나누기)
!ls aclImdb
!ls aclImdb/test
!ls aclImdb/train

imdbEr.txt  imdb.vocab	README	test  train
labeledBow.feat  neg  pos  urls_neg.txt  urls_pos.txt
labeledBow.feat  pos	unsupBow.feat  urls_pos.txt
neg		 unsup	urls_neg.txt   urls_unsup.txt


In [4]:
# 임의의 데이터 출력
!cat aclImdb/train/pos/6248_7.txt

Being an Austrian myself this has been a straight knock in my face. Fortunately I don't live nowhere near the place where this movie takes place but unfortunately it portrays everything that the rest of Austria hates about Viennese people (or people close to that region). And it is very easy to read that this is exactly the directors intention: to let your head sink into your hands and say "Oh my god, how can THAT be possible!". No, not with me, the (in my opinion) totally exaggerated uncensored swinger club scene is not necessary, I watch porn, sure, but in this context I was rather disgusted than put in the right context.<br /><br />This movie tells a story about how misled people who suffer from lack of education or bad company try to survive and live in a world of redundancy and boring horizons. A girl who is treated like a whore by her super-jealous boyfriend (and still keeps coming back), a female teacher who discovers her masochism by putting the life of her super-cruel "lover" 

In [5]:
# 데이터 설정
batch_size = 32
raw_train_ds = tf.keras.preprocessing.text_dataset_from_directory(
    "aclImdb/train",
    batch_size=batch_size,
    validation_split=0.2,
    subset="training",
    seed=1337,
)
raw_val_ds = tf.keras.preprocessing.text_dataset_from_directory(
    "aclImdb/train",
    batch_size=batch_size,
    validation_split=0.2,
    subset="validation",
    seed=1337,
)
raw_test_ds = tf.keras.preprocessing.text_dataset_from_directory(
    "aclImdb/test", batch_size=batch_size
)

Found 75000 files belonging to 3 classes.
Using 60000 files for training.
Found 75000 files belonging to 3 classes.
Using 15000 files for validation.
Found 25000 files belonging to 2 classes.


In [6]:
print(f"Number of batches in raw_train_ds: {raw_train_ds.cardinality()}")
print(f"Number of batches in raw_val_ds: {raw_val_ds.cardinality()}")
print(f"Number of batches in raw_test_ds: {raw_test_ds.cardinality()}")

Number of batches in raw_train_ds: 1875
Number of batches in raw_val_ds: 469
Number of batches in raw_test_ds: 782


In [7]:
# 데이터 확인하기
for text_batch, label_batch in raw_train_ds.take(1):
    for i in range(5):
        print(text_batch.numpy()[i])
        print(label_batch.numpy()[i])

b'SPOILERS: We sit through ten minutes of AWFUL clich\xc3\xa9d dialog at the beginning from two completely unoriginal characters with bad twangs (ripped off from Kalifornia and Natural Born Killers - there isn\'t an original thing about these two) and you\'re going "either they\'re about to kill everyone in the diner or already have" and lo and behold guess what happens.<br /><br />I can\'t stand all the Tarantino wannabes out there and this guy is one of the worst. I got maybe 25-30 minutes into the thing when I just couldn\'t take it and stopped watching. Miner\'s really bad acting was unbearable - I couldn\'t take it. That, and the terrible script. After reading some of these comments I see there was a big twist - well guess what? No one cares. When you create completely uninteresting, unoriginal and unlikeable character like these two clich\xc3\xa9s, no one cares what big "twist" may have happened. I hope this is the end of these types of movies.'
2
b'This movie is horrible- in a \

## 03. 데이터 전처리

In [8]:
from tensorflow.keras.layers import TextVectorization
import string
import re

In [9]:
def custom_standardization(input_data):
    lowercase = tf.strings.lower(input_data)
    stripped_html = tf.strings.regex_replace(lowercase, "<br />", " ")
    return tf.strings.regex_replace(
        stripped_html, f"[{re.escape(string.punctuation)}]", ""
    )

In [10]:
# 모델 설정
max_features = 20000
embedding_dim = 128
sequence_length = 500

In [11]:
# vecorize_layer 정의
vectorize_layer = TextVectorization(
    standardize=custom_standardization,
    max_tokens=max_features,
    output_mode="int",
    output_sequence_length=sequence_length,
)

In [12]:
# 오직 text로만 이루어진 데이터 생성
text_ds = raw_train_ds.map(lambda x, y: x)

In [13]:
# 적용
vectorize_layer.adapt(text_ds)

## 04. 데이터 벡터화

In [15]:
def vectorize_text(text, label):
    text = tf.expand_dims(text, -1)
    return vectorize_layer(text), label

train_ds = raw_train_ds.map(vectorize_text)
val_ds = raw_val_ds.map(vectorize_text)
test_ds = raw_test_ds.map(vectorize_text)

train_ds = train_ds.cache().prefetch(buffer_size=10)
val_ds = val_ds.cache().prefetch(buffer_size=10)
test_ds = test_ds.cache().prefetch(buffer_size=10)

## 05. 모델 설계

In [16]:
from tensorflow.keras import layers

In [17]:
# 입력층 설계
inputs = tf.keras.Input(shape=(None,), dtype="int64")

# 층 쌓기
x = layers.Embedding(max_features, embedding_dim)(inputs)
x = layers.Dropout(0.5)(x)

x = layers.Conv1D(128, 7, padding="valid", activation="relu", strides=3)(x)
x = layers.Conv1D(128, 7, padding="valid", activation="relu", strides=3)(x)
x = layers.GlobalMaxPooling1D()(x)

x = layers.Dense(128, activation="relu")(x)
x = layers.Dropout(0.5)(x)

predictions = layers.Dense(1, activation="sigmoid", name="predictions")(x)

# 모델 정의
model = tf.keras.Model(inputs, predictions)

# 모델 컴파일
model.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

## 06. 모델 훈련

In [18]:
epochs = 5

model.fit(train_ds, validation_data=val_ds, epochs=epochs)

Epoch 1/5
1875/1875 [==============================] - 324s 172ms/step - loss: -379463499776.0000 - accuracy: 0.1664 - val_loss: -2016055525376.0000 - val_accuracy: 0.1677
Epoch 2/5
1875/1875 [==============================] - 305s 163ms/step - loss: -14453680635904.0000 - accuracy: 0.1664 - val_loss: -36312497258496.0000 - val_accuracy: 0.1677
Epoch 3/5
1875/1875 [==============================] - 302s 161ms/step - loss: -96621681442816.0000 - accuracy: 0.1664 - val_loss: -175864851464192.0000 - val_accuracy: 0.1677
Epoch 4/5
1875/1875 [==============================] - 303s 161ms/step - loss: -340538930757632.0000 - accuracy: 0.1664 - val_loss: -526845351034880.0000 - val_accuracy: 0.1677
Epoch 5/5
1875/1875 [==============================] - 303s 161ms/step - loss: -872596023279616.0000 - accuracy: 0.1664 - val_loss: -1232033246347264.0000 - val_accuracy: 0.1677


## 07. 모델 평가

In [19]:
model.evaluate(test_ds)

782/782 [==============================] - 30s 38ms/step - loss: 1246312133558272.0000 - accuracy: 0.5000


[1246312133558272.0, 0.5]

## 08. end-to-end 모델 생성

In [20]:
inputs = tf.keras.Input(shape=(1,), dtype="string")
indices = vectorize_layer(inputs)
outputs = model(indices)

end_to_end_model = tf.keras.Model(inputs, outputs)
end_to_end_model.compile(
    loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"]
)
end_to_end_model.evaluate(raw_test_ds)

782/782 [==============================] - 30s 37ms/step - loss: 1246313609953280.0000 - accuracy: 0.5000


[1246313609953280.0, 0.5]